In [7]:
from pyspark import SparkContext,SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import Row, SparkSession
from pyspark.sql.types import IntegerType
from pyspark.sql import functions as F
from pyspark import SparkConf
import pandas as pd
import pickle
from tqdm import tqdm
# spark = SparkSession.builder.config('spark.executor.cores', '8').config('spark.executor.memory', '80g')\
#         .config("spark.sql.session.timeZone", "UTC").config('spark.driver.memory', '80g').master("local[26]")\
#         .appName("wego-daily").config('spark.driver.extraJavaOptions', '-Duser.timezone=UTC').config('spark.executor.extraJavaOptions', '-Duser.timezone=UTC')\
#         .config("spark.sql.datetime.java8API.enabled", "true").config("spark.sql.execution.arrow.pyspark.enabled", "true")\
#         .config("spark.sql.autoBroadcastJoinThreshold", -1)\
#         .config("spark.driver.maxResultSize", 0)\
#         .config("spark.shuffle.spill", "true")\
#         .config("spark.driver.extraJavaOptions", "-Dio.netty.tryReflectionSetAccessible=true")\
#         .config("spark.executor.extraJavaOptions", "-Dio.netty.tryReflectionSetAccessible=true")\
#         .config("spark.ui.showConsoleProgress", "false")\
#         .getOrCreate()
# spark.sparkContext.setLogLevel("ERROR")

In [8]:
import pandas as pd
import geopandas as gpd
from keplergl import KeplerGl
import json

/home/jptalusan/anaconda3/envs/py39/lib/python3.9/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [9]:
# # Load inrix segment data
# fp = '/home/jptalusan/wego-occupancy/data/backup_trip_traffic_df/inrix_speed_30M_resampled_2021_8.gz'
# traffic_estimate_df = pd.read_parquet(fp, engine='auto')

# fp = os.path.join('/home/jptalusan/wego-occupancy/data/inrix_grouped.pkl')
# with open(fp, "rb") as fh:
#   inrix_segment_df = pickle.load(fh)

# inrix_segment_df = inrix_segment_df.set_geometry('geometry')
# inrix_segment_df = inrix_segment_df[inrix_segment_df['County_inrix'] == 'davidson']
# # inrix_segment_df = inrix_segment_df.to_crs("EPSG:4326")
# traffic_estimate_df = traffic_estimate_df[traffic_estimate_df.measurement_tstamp.dt.day == 23]
# traffic_estimate_df = traffic_estimate_df.merge(inrix_segment_df[['geometry', 'XDSegID']], left_on='xd_id', right_on='XDSegID')
# traffic_estimate_df = gpd.GeoDataFrame(traffic_estimate_df, geometry='geometry')
# # traffic_estimate_df = traffic_estimate_df.to_crs("EPSG:3310")
# # traffic_estimate_df['geometry'] = traffic_estimate_df['geometry'].apply(lambda x: reverse_geom(x))
# traffic_estimate_df

In [10]:
fp = '/home/jptalusan/gits/mta_simulator_redo/code_root/scenarios/baseline/data/stops_node_matching.pkl'
stops_info = pd.read_pickle(fp)
stops_info.head(1)

,stop_sequence,stop_id_original,stop_name,map_latitude,map_longitude,nearest_node,nearest_edge
0,24,12ADEMNN,12TH AVE S & DEMONBREUN ST NB,36.154509,-86.786056,202342839,"(202284281, 202342839, 0)"


In [13]:
# Get service disruption dataset
fp = '/home/jptalusan/gits/mta_simulator_redo/data_generation/data/Service Disruptions_07_2019_08_2022.csv'
disruptions_df = pd.read_csv(fp)
disruptions_df.head()
disruptions_df['DATETIME'] = disruptions_df['DATE'] + ' ' + disruptions_df['TIME']
disruptions_df['DATE'] = pd.to_datetime(disruptions_df['DATE'], format='%m/%d/%y', errors='coerce')
disruptions_df['TIME'] = pd.to_datetime(disruptions_df['TIME'], format='%H:%M:%S', errors='coerce')
disruptions_df['DATETIME'] = pd.to_datetime(disruptions_df['DATETIME'], format='%m/%d/%y %H:%M:%S', errors='coerce')

# Remove weather related disruptions
# disruptions_df = disruptions_df[(disruptions_df['REASON'] != 'Weather')].sort_values(by=['DATETIME']).reset_index(drop=True)
print('Shape:', disruptions_df.shape)
# disruptions_df = disruptions_df.drop(columns=['COMMENTS'])
disruptions_df['BLOCK'] = disruptions_df['BLOCK'].astype('int32')

# Convert to spark dataframe for merging
# disruptions_sp = spark.createDataFrame(disruptions_df)
# disruptions_sp = disruptions_sp.withColumn("BLOCK", F.col("BLOCK").cast(IntegerType()))

Shape: (5066, 10)


In [14]:
# config = map_1.config
# with open('/home/jptalusan/gits/mta_simulator_redo/code_root/scenarios/baseline/output/config.json', 'w') as fp:
#     json.dump(config, fp)

In [18]:
fp = '../code_root/visualizer_1.csv'
viz_df = pd.read_csv(fp)
viz_df = viz_df.fillna(method='backfill').sort_values(by='time')
viz_df = pd.merge(viz_df, stops_info[['stop_id_original', 'map_latitude', 'map_longitude']], left_on='last_visited_stop', right_on='stop_id_original')
# viz_df = viz_df.drop_duplicates()

In [19]:
viz_df = viz_df.drop_duplicates()
viz_df.to_csv('visualizer_small.csv')

In [20]:
viz_df

,time,id,trip_id,last_visited_stop,value,fraction,icon,stop_id_original,map_latitude,map_longitude
0,2021-08-23 05:53:00,1200,244857.0,HICHICNN,0.0,0.0,car,HICHICNN,36.052637,-86.654878
1,2021-08-23 05:53:00,123,244857.0,HICHICNN,0.0,0.0,car,HICHICNN,36.052637,-86.654878
2,2021-08-23 05:53:00,HICHICNN,244857.0,HICHICNN,2.0,0.0,add-person,HICHICNN,36.052637,-86.654878
3,2021-08-23 05:54:00,1200,244857.0,HICHICNN,0.0,0.0,car,HICHICNN,36.052637,-86.654878
4,2021-08-23 05:54:00,123,244857.0,HICHICNN,2.0,0.0,car,HICHICNN,36.052637,-86.654878
...,...,...,...,...,...,...,...,...,...,...
3101,2021-08-23 17:17:14,1200,246300.0,SMRY,7.0,0.0,car,SMRY,35.993176,-86.526609
3102,2021-08-23 17:18:00,1200,246300.0,SMRY,7.0,0.0,car,SMRY,35.993176,-86.526609
3103,2021-08-23 17:19:37,1200,246300.0,SMRY,7.0,0.0,car,SMRY,35.993176,-86.526609
3104,2021-08-23 17:24:02,1200,246300.0,SMRY,7.0,0.0,car,SMRY,35.993176,-86.526609


In [ ]:
map_1 = KeplerGl(height=800)
map_1.add_data(viz_df, name='buses')
map_1.add_data(disruptions_df, name='disruptions')
with open('/home/jptalusan/gits/mta_simulator_redo/code_root/scenarios/baseline/output/config.json', 'r') as fp:
    config = json.load(fp)
map_1.config = config
    
# map_1

In [ ]:
# Here nashville shapes
from shapely.geometry import shape
df = pd.read_json('here.nashville.shapes.json')
df['geometry'] = df['geom'].apply(lambda x: shape(x))
df = df.drop(['_id', 'geom'], axis=1)
gdf = gpd.GeoDataFrame(df)
gdf = gdf.set_geometry('geometry')
gdf['geom_type'] = 'MultiLineString'
gdf.to_file("nashville_HERE.geojson", driver='GeoJSON')
gdf = gpd.read_file("nashville_HERE.geojson")
map_1 = KeplerGl(height=800)
map_1.add_data(gdf, name='here_shapes')
map_1